In [ ]:
import numpy as np
import gzip
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import nengo
from nengo.utils.ensemble import response_curves
import os
import imageio
%matplotlib inline

In [ ]:
def read_image_database(path='faces.npz'):
    images = []
    for file in os.listdir(path):
        images.append(imageio.imread(os.path.join(path,file), as_gray=True))
    mat = np.array(images, dtype=np.uint8)
    return mat
mat = np.load('faces.npz')['arr_0'].astype(np.float64)
def half_resolution(X,rep=2):
    for _ in range(rep):
        r,c = 2*(X.shape[0]//2), 2*(X.shape[1]//2)
        X = 0.25*(X[:r:2, :c:2] + X[1:r:2, 1:c:2] + X[1:r:2, :c:2] + X[:r:2, 1:c:2])
    return X
images_small = []
for i in range(mat.shape[0]):
    images_small.append(half_resolution(mat[i], 3))
mat = np.array(images_small)
X = (2.0 * mat/255.0-1.0)

N = mat.shape[0]
h = mat.shape[1]
w = mat.shape[2]
mat.shape

In [ ]:
X = X.reshape(N, w*h)
X_zero_mean = X-np.mean(X,axis=0)
X_cov = (X_zero_mean.T @ X_zero_mean)/(X.shape[0]-1)
D,V = np.linalg.eigh(X_cov)
D = D[::-1]
V = V.T[::-1,:]
V = V / np.linalg.norm(V,axis=0)

In [ ]:
NN = int(np.floor(np.sqrt(N)))
cmap = cm.get_cmap('gray')
fig, axs = plt.subplots(NN, NN, figsize=(5.5, 7))
for i in range(NN):
    for j in range(NN):
        axs[i, j].imshow(V[NN * i + j].reshape(h, w), vmin=-0.125, vmax=0.125, cmap=cmap)
        axs[i, j].set_xticks([])
        axs[i, j].set_yticks([])
fig.tight_layout(w_pad=0.0, h_pad=0.0)

In [ ]:
plt.subplot(1,2,1)
plt.plot(D)
plt.xlabel("nth eigenface")
plt.ylabel("eigenvalue")
plt.subplots_adjust(right=2)

plt.subplot(1,2,2)
plt.plot(D)
plt.xlim(-1, 60)
plt.xlabel("nth eigenface")
plt.ylabel("eigenvalue")
plt.title("zoomed in")

###### Since the magnitude of the eigenvalues to the corresponding eigenvectors indicates how important the eigenface is, we can take the first x eigenfaces where the magnitude of its eigenvalue is relatively high. Looking at the zoomed in graph on the right, we see that selecting only the first 20 eigenfaces or even the first 15 will suffice for our Encoders.

In [ ]:
def genRandomLinearCombinations(faceList, n):
    from random import sample
    combinations = []
    for _ in range(n):
        numFaces = np.random.randint(1,25)
        coefficients = np.random.randint(-10,10,numFaces)
        indcs = np.random.randint(0, len(faceList)-1, numFaces)
        faces = [faceList[i] for i in indcs]
        combinations.append(sum([c*f for c,f in zip(coefficients,faces)]))
    return np.array(combinations)

In [ ]:
numEigenfaces = 15
encoders = V[:numEigenfaces]
#numCombs = (h*w)-numEigenfaces
#combs = genRandomLinearCombinations(encoders, (h*w)-numEigenfaces)
#E = np.concatenate((encoders, combs))

In [ ]:
# want to keep the same ensemble and see how it represents faces
# if we reinitialze the ensemble for each input, then the curves
# are meaningless 
def Model(X, numNeu, d, E,synap, T, isPlot):
    lenX = len(X)
    populationCurves = []
    representations = []
    sts = []
    model = nengo.Network(seed=581)
    with model:
        ensA = nengo.Ensemble(n_neurons=numNeu,dimensions=d,encoders=E.T,normalize_encoders=False)
    for i in range(len(X)):
        with model:
            inp = nengo.Node(X[i])
            il = nengo.Connection(inp,ensA.neurons)
            inpProbe = nengo.Probe(inp)
            spikes = nengo.Probe(ensA.neurons)
            voltage = nengo.Probe(ensA.neurons, 'voltage')
            filtered = nengo.Probe(ensA,synapse=synap)
        with nengo.Simulator(model, progress_bar=False, optimize=True) as sim:
            if i%10==0: print("running simulation for step:",i)
            sim.run(T)
            # x_axis = 50 x vals
            # y_axis = n_neurons * 50 y values
            responseCurves = response_curves(ensA,sim)
            populationCurves.append(responseCurves)
            representations.append(responseCurves[1])
            sts.append(sim.data[filtered].T)
            if isPlot:
                plt.subplot(12,7,i+1)
                plt.plot(sim.trange(), sim.data[filtered])
    plt.show()
    return populationCurves, representations, sts

In [ ]:
def plotImages(X):
    NN = int(np.floor(np.sqrt(len(X))))
    fig,axs = plt.subplots(NN,NN,figsize=(5.5,7))
    for i in range(NN):
        for j in range(NN):
            axs[i,j].imshow(X[NN*i+j].reshape(h,w),cmap='gray')
            axs[i,j].set_xticks([])
            axs[i, j].set_yticks([])
    fig.tight_layout(w_pad=0.0,h_pad=0.0)

In [ ]:
def zeroHorizontal(X,s,e):
    for x in X:
        for i in range(s,e):
            x[i] = -1
    return X

## First get neural response for origianl face data

In [ ]:
curves1,reps1,st1 = Model(X=X,numNeu=h*w,d=15,E=encoders,synap=0.01,T=5,isPlot=0)

## Experiment 1: subtract mean face

We run this experiment to see if the neural response of our population will have less overlap, if the original inputs have less overlap. And depending on which output signals are better, we will use them to compare to the rest of our experiments.

In [ ]:
meanFace = np.mean(X,axis=0)
X1 = [x-meanFace for x in X]
plotImages(np.array(X).reshape(N,h,w))
plotImages(np.array(X1).reshape(N,h,w))

In [ ]:
curvesX1,repsX1,stX1 = Model(X=X1,numNeu=h*w,d=15,E=encoders,synap=0.01,T=5,isPlot=0)

## Experiment 2: drastically darkening skin colour

Will perform the following by deconstructing each face's PCA weights, and increasing the weight that corresponds to PC2, and reconstructing the face.

In [ ]:
# get weights [w1,w2,...,wn] where n is 2700, by computing [PCi * (Xi - meanFace)] for i in range(1,n)
# note we use all n weights to reconstruct the image after amplifying PC2, because we do not want to
# reduce image quality here, we just want to adjust a certain component of the image.
def getWeights(X, PCs, meanFace):
    weights = []
    for i in range(len(X)):
        w = PCs[i] * (X[i] - meanFace)
        weights.append(w)
    return weights

# i is the ith component we would like to darken
def darkenComponent(i,weights,factor):
    for w in weights:
        w[i] *= -factor if w[i]>0 else factor
        if w[i] < -1: w[i] = -1
            
def reconstructFaces(weights,PCs):
    faces = []
    for ws in weights:
        face = np.zeros(len(ws))
        for w,pc in zip(ws,PCs):
            face += w*pc
        faces.append(face)
    return faces
#weights = getWeights(X,V,meanFace)
#darkenComponent(1, weights, 1)
#X2 = reconstructFaces(weights,V)

In [ ]:
## ^ above reconstruction method did not work so we will just add more of a component to all our faces.
def changeComponent(change,i, PCs, originalFaces, factor):
    newFaces = []
    for f in originalFaces.reshape(N,h*w):
        newFace = f - factor*PCs[i] if change=="darken" else f +factor*PCs[i]
        newFaces.append(newFace)
    return newFaces
X2 = changeComponent("darken",0,V,X,15)
plotImages(np.array(X2).reshape(N,h,w))

In [ ]:
curvesX2,repsX2,stX2 = Model(X=X2,numNeu=h*w,d=15,E=encoders,synap=0.01,T=5,isPlot=0)

In [ ]:
X12 = changeComponent("darken",0,V,np.array(X1),10)
plotImages(np.array(X12).reshape(N,h,w))

In [ ]:
curvesX12,repsX12,stX12 = Model(X=X12,numNeu=h*w,d=15,E=encoders,synap=0.01,T=5,isPlot=0)

## Experiment 3: drastically brighten skin colour

In [ ]:
X3 = changeComponent("brighten",0,V,X,15)
plotImages(np.array(X3).reshape(N,h,w))

In [ ]:
curvesX3,repsX3,stX3 = Model(X=X3,numNeu=h*w,d=15,E=encoders,synap=0.01,T=5,isPlot=0)

In [ ]:
X13 = changeComponent("brighten",0,V,np.array(X1),15)
plotImages(np.array(X13).reshape(N,h,w))

In [ ]:
curvesX13,repsX13,stX13 = Model(X=X13,numNeu=h*w,d=15,E=encoders,synap=0.01,T=5,isPlot=0)